# Challenge 9 - Integrating an Azure ML Web Service

## Create Azure ML Web Service

1. In the Azure portal, create a new Azure Machine Learning Service workspace.
2. Open the Azure Machine Learning Visual Interface.
3. Create a new Dataset, uploading the insurance_claims.csv file.
4. Create a new blank experiment, and add the modules shown below:

  ![Training Experiment](Challenge09-Train.png)
  
5. Run the experiment, creating compute as required. Then review the output of the **Evaluate Model** module.
6. Create a *Predictive Experiment* from the training experiment.
7. Modify the default predictive experiment like this:

  ![Predictive Experiment](Challenge09-Predict.png)

8. Go back to the Azure ML workspace browser tab in the Portal and create AKS Compute to host your web service.
9. Deploy the predictive experiment as a web service to the compute you created.
10. Aftre the web service is created, view it in the Visual Interface and on its **Consume** tab; note the **Primary Key** and **Request-Response Url** values.
10. Modify and run the following Python code to test your web service.

In [1]:
import requests
from requests import post as http_post
import json

# REPLACE url and api_key WITH THE VALUES FOR YOUR WEB SERVICE
url = 'YOUR_AZUREML_SERVICE_URL'
api_key = 'YOUR_AZUREML_SERVICE_KEY'

body = {
  "Inputs": {
    "input1": [
      {
        "AgeOnDeparture": 52,
        "Destination": "dubai",
        "DepartureMonth": 12,
        "DurationOfStay_Days": 18
      }
    ]
  },
  "GlobalParameters": {}
}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

resp = http_post(url = url, data = json.dumps(body), headers = headers)

print(json.dumps(resp.json(), indent=2))


{
  "Results": {
    "output1": [
      {
        "claim_prediction": 1,
        "claim_probability": 0.9903456161957827
      }
    ]
  }
}


## Create an Azure Function

Create a Python Azure Function with the following code

```
import logging
import os
import sys
import json
import requests
from requests import post as http_post
from datetime import datetime
import azure.functions as func


def main(req: func.HttpRequest) -&gt; func.HttpResponse:
    logging.info('Python HTTP trigger function processed a request.')

    # The result will be a "values" bag
    result = {
        "values": []
    }
    statuscode = 200

    try:
        values = req.get_json().get('values')
        logging.info(values)

        for rec in values:
            # Construct the basic JSON response for this record
            val = {
                    "recordId": rec['recordId'],
                    "data": {
                        "claim_prediction":None,
                        "claim_probability":None
                    },
                    "errors": None,
                    "warnings": None
                }
            try:
                # get the data fields from the input record
                birth_date = rec['data']['birthdate']
                destination = rec['data']['destinationcity']
                departure_date = rec['data']['departuredate']
                return_date = rec['data']['returndate']

                # Calculate age on departure
                birth = datetime.strptime(birth_date, '%m/%d/%Y')
                departure = datetime.strptime(departure_date, '%m/%d/%Y')
                age = departure.year - birth.year

                # Calculate stay_duration
                lastday = datetime.strptime(return_date, '%m/%d/%Y')
                duration = (lastday - departure).days

                # REPLACE url and api_key WITH THE VALUES FOR YOUR WEB SERVICE
                model_service_url = 'YOUR_AZURE_ML_SERVICE_URL'
                api_key = 'YOUR_AZURE_ML_SERVICE_KEY'

                body = {
                "Inputs": {
                    "input1": [
                    {
                        "AgeOnDeparture": age,
                        "Destination": destination.lower(),
                        "DepartureMonth": departure.month,
                        "DurationOfStay_Days": duration
                    }
                    ]
                },
                "GlobalParameters": {}
                }

                headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

                resp = http_post(url = model_service_url, data = json.dumps(body), headers = headers)

                #print(json.dumps(resp.json(), indent=2))
                result_item = resp.json()["Results"]["output1"][0]
                val["data"]["claim_prediction"] =result_item["claim_prediction"]
                val["data"]["claim_probability"] =result_item["claim_probability"]
                
            except:
                # An error occured for this text record, so add lists of errors and warning
                val["errors"] =[{"message": "An error occurred processing the content."}]
                val["warnings"] = [{"message": "One or more inputs failed to process."}]
            finally:
                # Add the value for this record to the response
                result["values"].append(val)
    except Exception as ex:
        statuscode = 500
        # A global error occurred, so return an error response
        val = {
                "recordId": None,
                "data": {
                    "text":None
                },
                "errors": [{"message": ex.args}],
                "warnings": [{"message": "The request failed to process."}]
            }
        result["values"].append(val)
    finally:
        # Return the response
        return func.HttpResponse(body=json.dumps(result), mimetype="application/json", status_code=statuscode)

```

Add `requests` to **requirements.txt**:


In [4]:
# Test the function

from requests import post as http_post
import json
import base64

# URL for the Azure Function
func = "YOUR_AZURE_FUNCTION_ENDPOINT"


# Send a request in the same format as the Azure Search service custom skill

body = {
        "values": [
            {
                "recordId": "a1",
                "data":
                {
                    "birthdate": "07/03/1971",
                    "destinationcity": "San Francisco",
                    "departuredate": "11/19/2018",
                    "returndate": "11/25/2018"
                }
            },
            {
                "recordId": "a2",
                "data":
                {
                    "birthdate": "03/03/1979",
                    "destinationcity": "Las Vegas",
                    "departuredate": "11/12/2018",
                    "returndate": "11/25/2018"
                }
            }
       ]
    }

headers = {
    # Request headers
    'Content-Type': 'application/json'
}

print("Request")
print(json.dumps(body, indent=2))

resp = http_post(url = func, data = json.dumps(body), headers = headers)
print("Response status code: %d" % resp.status_code)
print(json.dumps(resp.json(), indent=2))

Request
{
  "values": [
    {
      "recordId": "a1",
      "data": {
        "birthdate": "07/03/1971",
        "destinationcity": "San Francisco",
        "departuredate": "11/19/2018",
        "returndate": "11/25/2018"
      }
    },
    {
      "recordId": "a2",
      "data": {
        "birthdate": "03/03/1979",
        "destinationcity": "Las Vegas",
        "departuredate": "11/12/2018",
        "returndate": "11/25/2018"
      }
    }
  ]
}
Response status code: 200
{
  "values": [
    {
      "recordId": "a1",
      "data": {
        "claim_prediction": 0,
        "claim_probability": 0.013985217614474568
      },
      "errors": null,
      "warnings": null
    },
    {
      "recordId": "a2",
      "data": {
        "claim_prediction": 1,
        "claim_probability": 0.7713189815661405
      },
      "errors": null,
      "warnings": null
    }
  ]
}


## Update Azure Search Index

In [5]:
# Azure Search constants
azsearch_url = "YOUR_SEARCH_SERVICE.search.windows.net"
azsearch_key = "YOUR_AZURE_SEARCH_KEY"

# Cognitive Services Key
cog_svc_key = "YOUR_COGNITIVE_SERVICES_KEY"

# Function URLs
form_function_url = "YOUR_FORM_RECOGNITION_FUNCTION_ENDPOINT (FROM PREVIOUS CHALLENGE)"
claim_function_url = "YOUR_CLAIM_PREDICTION_FUNCTION_ENDPOINT"

In [6]:
## Helper function for calling Azure Search API

def azsearch_rest(request_type="GET", function_name="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2019-05-06-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(function_name, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API")

Ready to use the REST API


In [7]:
# Verify data source exists
import json

try:
    result = azsearch_rest(function_name="datasources")
    if len(result) > 0:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

200
{
  "@odata.context": "https://margies-search.search.windows.net/$metadata#datasources",
  "value": [
    {
      "@odata.etag": "\"0x8D724EA69A47DBB\"",
      "container": {
        "name": "margies-forms",
        "query": null
      },
      "credentials": {
        "connectionString": null
      },
      "dataChangeDetectionPolicy": null,
      "dataDeletionDetectionPolicy": null,
      "description": "forms.",
      "name": "internal-forms",
      "subtype": null,
      "type": "azureblob"
    },
    {
      "@odata.etag": "\"0x8D71B7C6014DCE1\"",
      "container": {
        "name": "margies-data",
        "query": null
      },
      "credentials": {
        "connectionString": null
      },
      "dataChangeDetectionPolicy": null,
      "dataDeletionDetectionPolicy": null,
      "description": "Various documents.",
      "name": "website-docs",
      "subtype": null,
      "type": "azureblob"
    }
  ]
}


In [8]:
# Update the index

import json

# Define the request body
body = {
  "fields": [
    {
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "sortable": True
    },
    {
      "name": "url",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "file_name",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "size",
      "type": "Edm.Int64",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "last_modified",
      "type": "Edm.DateTimeOffset",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "content",
      "type": "Edm.String",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "name",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "address",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "city",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "post_code",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "country",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "birth_date",
      "type": "Edm.String",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "destination_city",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "destination_country",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "departure_date",
      "type": "Edm.String",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "return_date",
      "type": "Edm.String",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "policy_date",
      "type": "Edm.String",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    },
      ## New Fields for ML results
    {
      "name": "claim_prediction",
      "type": "Edm.Int64",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    },
    {
      "name": "claim_probability",
      "type": "Edm.Double",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    }
  ]
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexes/form-recognizer-index", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

204


In [9]:
# Modify Skillset
# Define the request body
body = {
  "description": "Extract and enhance text",
  "skills":
  [
      # Use a custom skill to call form recognizer function
          {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "custom form recognizer skill",
            "uri": form_function_url,
            "batchSize":1,
            "context": "/document",
            "inputs": [
              {
                "name": "url",
                "source": "/document/url"
              },
              {
                "name": "sas_token",
                "source": "/document/metadata_storage_sas_token"
              }
            ],
            "outputs": [
              {
                "name": "name:",
                "targetName": "name"
              },
              {
                "name": "address:",
                "targetName": "address"
              },
              {
                "name": "city:",
                "targetName": "city"
              },
              {
                "name": "postalcode:",
                "targetName": "post_code"
              },
              {
                "name": "country:",
                "targetName": "country"
              },
              {
                "name": "dateofbirth:",
                "targetName": "birth_date"
              },
              {
                "name": "destinationcity:",
                "targetName": "destination_city"
              },
              {
                "name": "destinationcountry:",
                "targetName": "destination_country"
              },
              {
                "name": "departuredate:",
                "targetName": "departure_date"
              },
              {
                "name": "returndate:",
                "targetName": "return_date"
              },
              {
                "name": "date:",
                "targetName": "policy_date"
              }
            ]
          },
           # Use a custom skill to call ML web service function
          {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "custom Azure ML skill",
            "uri": claim_function_url,
            "batchSize":1,
            "context": "/document",
            "inputs": [
              {
                "name": "birthdate",
                "source": "/document/birth_date"
              },
              {
                "name": "destinationcity",
                "source": "/document/destination_city"
              },
              {
                "name": "departuredate",
                "source": "/document/departure_date"
              },
              {
                "name": "returndate",
                "source": "/document/return_date"
              }
            ],
            "outputs": [
              {
                "name": "claim_prediction",
                "targetName": "claim_prediction"
              },
              {
                "name": "claim_probability",
                "targetName": "claim_probability"
              }]
          }
      ],
      "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "margies-cog-svc",
        "key": cog_svc_key
      }
    }  

try:
    result = azsearch_rest(request_type="PUT", function_name="/skillsets/form-recognizer-skillset", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


In [10]:
# Update the indexer
import json

body = {
  "name":"form-recognizer-indexer", 
  "dataSourceName" : "internal-forms",
  "skillsetName" : "form-recognizer-skillset",
  "targetIndexName" : "form-recognizer-index",
  "fieldMappings" : [
      # Map the soure content fields to index fields
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "id",
          "mappingFunction" : 
            { "name" : "base64Encode" }
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "url"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "file_name"
        },
        {
          "sourceFieldName" : "content",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "metadata_storage_size",
          "targetFieldName" : "size"
        },
        {
          "sourceFieldName" : "metadata_storage_last_modified",
          "targetFieldName" : "last_modified"
        }
   ],
  "outputFieldMappings" : 
  [
      # Map the output from custom skill to index fields
        {
          "sourceFieldName" : "/document/name", 
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "/document/address", 
          "targetFieldName" : "address"
        },
        {
          "sourceFieldName" : "/document/city", 
          "targetFieldName" : "city"
        },
        {
          "sourceFieldName" : "/document/post_code", 
          "targetFieldName" : "post_code"
        },
        {
          "sourceFieldName" : "/document/country", 
          "targetFieldName" : "country"
        },
        {
          "sourceFieldName" : "/document/birth_date", 
          "targetFieldName" : "birth_date"
        },
        {
          "sourceFieldName" : "/document/destination_city", 
          "targetFieldName" : "destination_city"
        },
        {
          "sourceFieldName" : "/document/destination_country", 
          "targetFieldName" : "destination_country"
        },
        {
          "sourceFieldName" : "/document/departure_date", 
          "targetFieldName" : "departure_date"
        },
        {
          "sourceFieldName" : "/document/return_date", 
          "targetFieldName" : "return_date"
        },
        {
          "sourceFieldName" : "/document/policy_date", 
          "targetFieldName" : "policy_date"
        },
        {
          "sourceFieldName" : "/document/claim_prediction", 
          "targetFieldName" : "claim_prediction"
        },
        {
          "sourceFieldName" : "/document/claim_probability", 
          "targetFieldName" : "claim_probability"
        }
  ],
  "parameters":
  {
    "maxFailedItems":-1,
    "maxFailedItemsPerBatch":-1,
    "configuration": 
    {
        "dataToExtract": "contentAndMetadata",
        "imageAction": "generateNormalizedImages"
    }
  }
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexers/form-recognizer-indexer", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


In [11]:
# Rerun the idenxer
import time

try:
    # Reset the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/form-recognizer-indexer/reset", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
        
    # re-run the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/form-recognizer-indexer/run", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
    


except Exception as e:
    print('Error:')
    print(e)

204
202


In [12]:
# Get Indexer status until complete

import time, json

try:
    complete = False
    while (complete == False):
        result = azsearch_rest(request_type="GET", function_name="indexers/form-recognizer-indexer/status")
        state = result["status"]
        if result['lastResult'] is not None:
            state = result['lastResult']['status']
        print (state)
        if state in ("success", "error"):
            complete = True
        time.sleep(1)

except Exception as e:
    print('Error:')
    print(e)

200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
success


## Search the Index

In [13]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


In [14]:
import urllib.parse, json

search_terms = input("Search?\n")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, name, address, city, post_code, birth_date, destination_city, destination_country, departure_date, return_date, policy_date, claim_probability, claim_prediction',
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="form-recognizer-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    
    for value in result["value"]:
        print(value["name"], "from", value["city"], "travelling to", value["destination_city"], "on", value["departure_date"])
        print("Claim probability:",value["claim_probability"],"\n")
except Exception as e:
    print('Error:')
    print(e)

Search?
London
Hits: 2
Josefina Conrad from Los Angeles travelling to London on 10/1/2018
Claim probability: 0.9851928033370134 

Issac Collet from Seattle travelling to London on 01/15/2018
Claim probability: 0.06626456288777804 



In [15]:
import urllib.parse, json

search_terms = input("Search?\n")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, name, address, city, post_code, birth_date, destination_city, destination_country, departure_date, return_date, policy_date, claim_probability, claim_prediction',
    '$filter': "claim_probability ge 0.5",
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="form-recognizer-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    
    for value in result["value"]:
        print(value["name"], "from", value["city"], "travelling to", value["destination_city"], "on", value["departure_date"])
        print("Claim probability:",value["claim_probability"],"\n")
except Exception as e:
    print('Error:')
    print(e)

Search?
London
Hits: 1
Josefina Conrad from Los Angeles travelling to London on 10/1/2018
Claim probability: 0.9851928033370134 

